In [1]:
import pandas as pd
import sqlite3

In [2]:
%run ./transform_and_create_tables.ipynb

<class 'str'>


/tmp/ipykernel_74381/3054090517.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  base_originacao.VLCONTRATO = base_originacao.VLCONTRATO.str.replace('.', '').str.replace(',', '.').apply(lambda x: float(x))
/tmp/ipykernel_74381/3054090517.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  base_originacao.VLPARCELA = base_originacao.VLPARCELA.str.replace('.', '').str.replace(',', '.').apply(lambda x: float(x))
/tmp/ipykernel_74381/3054090517.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  base_carteira.VLCONTRATO =

contém 0 registros duplicados na base_carteira!
contém 0 registros duplicados na base_originacao!


In [3]:
conn = sqlite3.connect('teste_ailos.db')

base_originacao = pd.read_sql_query("SELECT * FROM base_originacao", conn)
base_carteira = pd.read_sql_query("SELECT * FROM base_carteira", conn)

In [4]:
%reload_ext sql
%sql sqlite:///teste_ailos.db

In [5]:
base_originacao.head()

,index,COOPERATIVA,CPF,CONTA,CONTRATO,DSPRODUTO,DTCONTRATO,VLCONTRATO,VLPARCELA,QTPARCELAS
0,0,A,32,496314,677366,Financiamento,2021-10-13 00:00:00,4263.6,153.0,36.0
1,1,A,38,624412,644724,Empréstimo,2021-09-01 00:00:00,81.6,10.2,10.0
2,2,A,154,49834,349384,Empréstimo,2020-08-25 00:00:00,9761.4,204.0,72.0
3,3,A,186,398914,444074,Empréstimo,2021-01-08 00:00:00,4314.6,163.2,36.0
4,4,A,430,549582,372569,Empréstimo,2020-10-02 00:00:00,5548.8,183.6,48.0


In [6]:
base_carteira.head()

,index,DTBASE,COOPERATIVA,CPF,CONTA,CONTRATO,DSPRODUTO,DTCONTRATO,VLCONTRATO,SDDEVEDOR,QTDIASATRASO,SDATRASO,VLPROVISAO
0,0,2021-01-31 00:00:00,A,154,49834,349384,Empréstimo,2020-08-25 00:00:00,9761.4,9557.4,0.0,0.0,30.6
1,1,2021-01-31 00:00:00,A,186,398914,444074,Empréstimo,2021-01-08 00:00:00,4314.6,4365.6,0.0,0.0,81.6
2,2,2021-01-31 00:00:00,A,430,549582,372569,Empréstimo,2020-10-02 00:00:00,5548.8,5273.4,0.0,0.0,20.4
3,3,2021-01-31 00:00:00,A,512,217818,223696,Empréstimo,2020-01-31 00:00:00,3111.0,2539.8,0.0,0.0,10.2
4,4,2021-01-31 00:00:00,A,754,546819,435007,Empréstimo,2020-12-23 00:00:00,601.8,601.8,19.0,601.8,10.2


# RESOLUÇÃO DO EXERCÍCIO EM SQL

### Para a Cooperativa A, obtenha a relação dos 10 CPFs com maiores Saldos Devedor para a data referência 30/09/2021, em que os CPFs apresentam algum atraso maior que zero

In [7]:
%%sql
select 
    CPF, round(sum(SDDEVEDOR),2) as sum_saldo_devedor
from 
    base_carteira
where
    SDATRASO > 0 
    and date(DTBASE) = '2021-09-30'
    and COOPERATIVA = 'A'
group by 
    CPF
order by 
    2 desc
limit 10;

 * sqlite:///teste_ailos.db
Done.


CPF,sum_saldo_devedor
80770,58425.6
565660,40269.6
714760,18421.2
127329,17941.8
164078,16697.4
286984,16442.4
629355,16411.8
265230,15759.0
409432,14137.2
334457,14055.6


# RESOLUÇÃO DO EXERCÍCIO EM PYTHON

### Para a Cooperativa A, obtenha a relação dos 10 CPFs com maiores Saldos Devedor para a data referência 30/09/2021, em que os CPFs apresentam algum atraso maior que zero

In [8]:
top_10_sddevedor = base_carteira[
                                    (base_carteira.DTBASE == '2021-09-30 00:00:00') & 
                                    (base_carteira.SDATRASO > 0) &
                                    (base_carteira.COOPERATIVA == 'A')].groupby('CPF')['SDDEVEDOR'].sum()

top_10_sddevedor.sort_values(ascending=False)[:10]

CPF
80770     58425.6
565660    40269.6
714760    18421.2
127329    17941.8
164078    16697.4
286984    16442.4
629355    16411.8
265230    15759.0
409432    14137.2
334457    14055.6
Name: SDDEVEDOR, dtype: float64